In [1]:
#!/usr/bin/env python
"""
CEVAE moargs.latent_dimensionel on IHargs.latent_dimensionP
"""
##################### Neeargs.latent_dimension TF1
# import edward as ed
# import tensorflow as tf
# from utils import fc_net, get_y0_y1
# from edward.models import Bernoulli, Normal
# from progressbar import ETA, Bar, Percentage, ProgressBar
###############################################################
import os
# import json
# import time
# import math
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.stats import sem
import tensorflow.keras as tfk
import tensorflow_probability as tfp
import tensorflow.keras.layers as tfkl
tfd,tfpl = tfp.distributions,tfp.layers
import tensorflow.keras.backend as tfkb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from datasets import IHDP
from evaluation import Evaluator
from argparse import ArgumentParser

parser = ArgumentParser()
################### CEVAE
parser.add_argument('-reps', type=int, default=10)
parser.add_argument('-earl', type=int, default=10)
parser.add_argument('-lr', type=float, default=0.001)
parser.add_argument('-decay', type=float, default=0.001)
parser.add_argument('-opt', choices=['adam', 'adamax'], default='adam')
parser.add_argument('-epochs', type=int, default=100)     
parser.add_argument('-print_every', type=int, default=10)
################### BASIC
parser.add_argument('--num_parallel_reads', type = int, default = 10, help = 'random seed')
parser.add_argument('--random_seed', type = int,default = 9418, help = 'random seed')
parser.add_argument('--mode', type = str, default = 'train', help = 'train or evaluate')
args = parser.parse_args([])
args.true_post = True
#################################From CEVAE
dataset = IHDP(replications=args.reps)
dimx = 25
scores = np.zeros((args.reps, 3))
scores_test = np.zeros((args.reps, 3))
#################################IHDP Data
# data information 
t_bin_dim = 1
y_dim, default_y_scale = 1,tf.exp(0.)
M = None        # batch size during training
z_dim = 20          # latent z dimension
lamba = 1e-4    # weight decay
nh, h = 3, 200  # number and size of hidden layers
binfeats = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
numfeats = [i for i in range(25) if i not in binfeats]
x_bin_dim = len(binfeats)
x_num_dim = len(numfeats)
################################# some functions needed
def findfile(start, name):
    ans = []
    for relpath, dirs, files in os.walk(start):
        for file in files:
            if name in file:
                full_path = os.path.join(start, file)
                ans.append(full_path)
                # print(os.path.normpath(os.path.abspath(full_path)))
    return ans

2022-02-23 13:19:18.225640: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# # create datasets
# data_all = []
# for data in findfile("./datasets/IHDP/csv",'.csv'):
#     try:
#         data_all = data_all.append(pd.read_csv(data, header = None), ignore_index = True)
#     except:
#         data_all = pd.read_csv(data, header = None)
# data_all.columns = ['treatment', 'y_factual', 'y_cfactual', 'mu0', 'mu1', ] + ['x'+str(i) for i in range(25)]
# y_mean,y_std = np.mean(data_all['y_factual']),np.std(data_all['y_factual'])
# data_all['y_factual'] = (data_all['y_factual'] - y_mean) / y_std
# data_all['x13'] = data_all['x13'] - 1
# # set random seeds:
# np.random.seed(7)
# data_train, data_valid = train_test_split(data_all, test_size=0.2, )
# data_train.to_csv("./datasets/IHDP/csv_create/train.csv", header = None, index =None)
# data_valid.to_csv("./datasets/IHDP/csv_create/valid.csv", header = None, index =None)

In [3]:
def ihdp_fn(data):
    data = tf.strings.to_number(tf.strings.split(data, sep=','))
    t, y, y_cf = data[0:1], data[1:2], data[2:3]
    mu_0, mu_1, x = data[3:4], data[4:5], data[5:]
    x_bin = x[..., x_num_dim:]  # binary inputs
    x_num = x[..., 0:x_num_dim]  # continuous inputs
    return [x_bin, x_num, t, y, y_cf, mu_0, mu_1]
    
def input_fn_csv(path,name, batch_size=128, shuffle_size=0):
    all_files = findfile(path,name)
    ds = tf.data.TextLineDataset(
        all_files,
        compression_type= None,
        num_parallel_reads=int(args.num_parallel_reads),
        buffer_size=64*1024*1024# 可选参数，多线程并行读多个文件
    ).repeat(args.reps)
    if shuffle_size > 0:
        ds = ds.shuffle(shuffle_size)
    ds = ds.map(ihdp_fn).batch(batch_size)
    return ds

In [4]:
from networks import p_x_z, p_t_z, p_y_tz, q_t_x, q_y_xt, q_z_xyt
class CEVAE(tf.keras.Model):
    def __init__(self):
        super(CEVAE, self).__init__()
        ########################################
        # networks
        self.activation = 'elu'
        # CEVAE Model (decoder)
        # p(z)
        self.z = tfd.Normal(loc = [0]*z_dim, scale = [1]*z_dim)
        # p(x|z)
        self.p_x_z = p_x_z(x_bin_dim, x_num_dim, z_dim, nh, h)
        # p(t|z)
        self.p_t_z = p_t_z(t_bin_dim, z_dim,h)
        # p(y|t,z)
        self.p_y_tz = p_y_tz(y_dim, t_bin_dim, z_dim,default_y_scale, nh, h )
        # CEVAE Model (encoder)
        # q(t|x)
        self.q_t_x = q_t_x(x_bin_dim, x_num_dim, t_bin_dim, z_dim, nh, h)
        # q(y|x,t)
        self.q_y_xt = q_y_xt(x_bin_dim, x_num_dim, y_dim, t_bin_dim, default_y_scale, nh, h)
        # q(z|x,t,y)
        self.q_z_xyt = q_z_xyt(x_bin_dim, x_num_dim, y_dim, t_bin_dim, z_dim, nh, h) 
        ########################################
        # optimizers
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            args.lr,
            decay_steps=1,
            decay_rate=args.decay,
            staircase=True)

        self.optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule)

    def call(self, data, training=False, serving=False):
        # Dataset_inp
        # input data preprocess
        [x_ph_bin, x_ph_num, t, y, _, _, _] = data
        x_ph = tf.concat([x_ph_bin, x_ph_num], -1)
        # CEVAE variational approximation (encoder)
        ## q(t|x)
        qt = self.q_t_x(x_ph)
        ## q(y|x,t)
        xt_inputs = tf.concat([x_ph,qt],-1)
        qy = self.q_y_xt(xt_inputs)
        ## q(z|x,t,y)
        xyt_input = tf.concat([x_ph, qy, qt], axis=-1)
        qz = self.q_z_xyt(xyt_input)

        # CEVAE model (decoder)
        ## p(x|z)
        [x_infer_bin,x_infer_num] = self.p_x_z(qz)
        ## p(t|z)
        t_infer = self.p_t_z(qz)
        ## p(y|t,z)
        y_infer = self.p_y_tz(tf.concat([t_infer,qz],-1))
        
        output = [x_infer_bin,x_infer_num,t_infer,y_infer, qz,qt,qy,]
        return output

    def cevae_loss(self,data,pred):
        # get preds
        [x_bin,x_num,t,y,qz,qt,qy] = pred
        # read labels
        [x_bin_real,x_num_real, t_real, y_real, y_cf_real, mu_0_real, mu_1_real] = data
        # Reconstruction loss
        ## p(x|z)
        loss = {}
        loss['loss_p_x_z_bin'] = tfkb.mean(x_bin.log_prob(x_bin_real))
        loss['loss_p_x_z_num'] = tfkb.mean(x_num.log_prob(x_num_real))
        ## p(t|z)
        loss['loss_p_t_z_bin'] = tfkb.mean(t.log_prob(t_real))
        ## p(y|t,z)
        loss['loss_p_y_tz'] = tfkb.mean(y.log_prob(y_real))
        # REGULARIZATION LOSS
        ## p(z) - q(z|x,t,y)
        ## approximate KL
        ### calculate based on sample returns none value
        # reg_loss_q_z = tfkb.mean(self.z.log_prob(qz.sample()) - qz.log_prob(qz.sample()))
        ### try use Analytic KL (seems to make overall performance less stable)
        # reg_loss_q_z = tfkb.mean(-tf.math.log(qz.stddev) + 1/2*(qz.variance + qz.mean**2 - 1))
        # tf.print(reg_loss_q_z)
        # AUXILIARY LOSS
        ## q(t|x)
        loss['aux_loss_q_t_x'] = tfkb.mean(qt.log_prob(t_real))
        ## q(y|x,t)
        loss['aux_loss_q_y_xt'] = tfkb.mean(qy.log_prob(y_real))
        
        loss_all = loss['loss_p_x_z_bin'] + loss['loss_p_x_z_num'] + loss['loss_p_t_z_bin'] + loss['loss_p_y_tz'] + loss['aux_loss_q_t_x'] + loss['aux_loss_q_y_xt']
        
        # tf.print(loss)
        return -loss_all

    def get_y0_y1(self, x_train, t_train, L=1):
        y_infer = self.q_y_xt(tf.concat([x_train, t_train],-1))
        # use inferred y
        xyt = tf.concat([x_train, y_infer, t_train], -1)  # TODO take mean?
        z_infer = self.q_z_xyt(xyt)
        # Manually input zeros and ones
        y0 = self.p_y_tz(tf.concat([t_train,tf.zeros_like(z_infer)],-1)).mean()  # TODO take mean?
        y1 = self.p_y_tz(tf.concat([t_train,tf.ones_like(z_infer)],-1)).mean()  # TODO take mean?
        return y0,y1

    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        with tf.GradientTape() as tape:
            pred = self(data, training=True)  # Forward pass
            # Compute the loss value.
            # The loss function is configured in `compile()`.
            loss = self.cevae_loss(data,pred)
        # Compute gradients
        gradients = tape.gradient(loss, self.trainable_weights)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        self.optimizer.lr.assign(args.lr)
        ##########################################################
        #  metrics
        [x_bin_real,x_num_real, t_real, y_real, y_cf, mu0, mu1] = data
        evaluate_metric = Evaluator(y = y_real, t = t_real, y_cf=y_cf, mu0=mu0, mu1=mu1)
        metrics = {"loss":loss, }
        # Compute our own metrics
        x_real = tf.concat([x_bin_real,x_num_real],-1)
        y0, y1 = self.get_y0_y1(x_real,t_real)
        score_train = evaluate_metric.calc_stats(y1, y0)
        rmses_train = evaluate_metric.y_errors(y0, y1)
        metrics.update({
            "train ite": score_train[0], 
            "train ate": score_train[1],
            "train pehe": score_train[2], 
            "train rmse":rmses_train[0],
            "train rmse counterfactual":rmses_train[1],
            })
        return metrics
    
    def test_step(self, data):
        ##########################################################
        #  metrics
        [x_bin_real,x_num_real, t_real, y_real, y_cf, mu0, mu1] = data
        evaluate_metric = Evaluator(y = y_real, t = t_real, y_cf=y_cf, mu0=mu0, mu1=mu1)
        metrics = {}
        # Compute our own metrics
        x_real = tf.concat([x_bin_real,x_num_real],-1)
        y0, y1 = self.get_y0_y1(x_real,t_real)
        score_train = evaluate_metric.calc_stats(y1, y0)
        rmses_train = evaluate_metric.y_errors(y0, y1)
        metrics.update({
            "valid ite": score_train[0], 
            "valid ate": score_train[1],
            "valid pehe": score_train[2], 
            "valid rmse":rmses_train[0],
            "valid rmse counterfactual":rmses_train[1],
            })
        return metrics

def build_model():
    model = CEVAE()
    model.compile(optimizer=tf.keras.optimizers.Adam(args.lr))
    return model

In [5]:
model = build_model()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="fit_logs/", histogram_freq=1)
train_dataset = input_fn_csv("./datasets/IHDP/csv_create",'train')
valid_dataset = input_fn_csv("./datasets/IHDP/csv_create",'valid')
model.fit(
    train_dataset,
    validation_data = valid_dataset, 
    callbacks=[tensorboard_callback],
    epochs = args.epochs,
    )

2022-02-23 13:19:18.524804: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 1/100
467/467 [==============================] - 12s 18ms/step - loss: 2.4557 - train ite: 9.8693 - train ate: 4.7819 - train pehe: 10.1258 - train rmse: 0.9565 - train rmse counterfactual: 19.1816 - val_valid ite: 9.6352 - val_valid ate: 4.1818 - val_valid pehe: 10.1445 - val_valid rmse: 0.9948 - val_valid rmse counterfactual: 16.8580
Epoch 2/100
467/467 [==============================] - 11s 24ms/step - loss: -0.9927 - train ite: 9.8421 - train ate: 4.7330 - train pehe: 10.1016 - train rmse: 0.9524 - train rmse counterfactual: 19.1389 - val_valid ite: 9.6249 - val_valid ate: 4.1272 - val_valid pehe: 10.1221 - val_valid rmse: 0.9927 - val_valid rmse counterfactual: 16.8468
Epoch 3/100
467/467 [==============================] - 13s 28ms/step - loss: -0.4211 - train ite: 9.8411 - train ate: 4.7008 - train pehe: 10.0856 - train rmse: 0.9524 - train rmse counterfactual: 19.1443 - val_valid ite: 9.6102 - val_valid ate: 4.1296 - val_valid pehe: 10.1231 - val_valid rmse: 0.9937 - val_v